# Scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe

# Scrape the Wikipedia page

In [2]:
import requests
import pandas as pd


wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wikipedia_link)
wikipedia_doc = wikipedia_page.text

# BeautifulSoup for reading in the table of postal code

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wikipedia_doc, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
#table

# Convert the table into a pandas dataframe

In [4]:
col_names = ["PostalCode", "Borough", "Neighborhood"]
df = pd.read_html(str(table), skiprows=1)
df= pd.DataFrame.from_dict(df[0])
df.columns = col_names
df.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


# Ignore cells with a borough that is Not assigned

In [5]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# If there is more than one neighborhood existing in one postal code area these neighborhoods will be combined into one row with the neighborhoods separated with a comma

In [6]:
def combine_neighborhood(series):
    return series.str.cat(sep=', ')

df_by_pcode = df.groupby(["PostalCode", "Borough"])
df = df_by_pcode.agg({'Neighborhood': combine_neighborhood}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [9]:
def impute_neirghborhood(row):
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    
    return row

df = df.apply(impute_neirghborhood, axis=1)
df.head(50)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# Print the number of rows of my dataframe

In [40]:
df.shape

(103, 3)